In [ ]:
import geopandas as gpd
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
# Read the India shapefile and see the attributes in the data
shapefile_path = r"\Data\shapefile\India-State-and-Country-Shapefile-Updated-Jan-2020-master\India_State_Boundary.shp" 
shape = gpd.read_file(shapefile_path)
# Get the list of the states 
shape

DataSourceError: '\Data\shapefile\India-State-and-Country-Shapefile-Updated-Jan-2020-master\India_State_Boundary.shp' does not exist in the file system, and is not recognized as a supported dataset name.

In [4]:
import xarray as xr
import rioxarray
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

# File paths
file_dir = r"Data\tmax\tmax.nc" 
shapefile_path = r"India_State_Boundary.shx"  

# Load dataset
data = xr.open_dataset(file_dir, engine='netcdf4', use_cftime=True)
temperature = data["tmax"]

# Convert to a format compatible with raster masking
temperature = temperature.rio.write_crs("EPSG:4326")  # Assuming WGS84 CRS

# Load shapefile and filter the state
all_states = gpd.read_file(shapefile_path)
state_name = "Odisha"  # 🔁 Change to the state you want
state = all_states[all_states["State_Name"] == state_name]  # make sure column name matches your shapefile

# Reproject the shapefile
region_gdf = state.to_crs("EPSG:4326")

# Get latest year
latest_year = pd.to_datetime(str(temperature.time.values[-1])).year
latest_year_data = temperature.sel(time=str(latest_year))

# Clip to shapefile region
region_latest = latest_year_data.rio.clip(region_gdf.geometry.values, region_gdf.crs, drop=True)

# Reference period
ref_period = temperature.sel(time=slice("1991-01-01", "2020-12-31"))
ref_region = ref_period.rio.clip(region_gdf.geometry.values, region_gdf.crs, drop=True)

# Compute mean and std from reference period
mean_ref = ref_region.mean(dim="time")
std_ref = ref_region.std(dim="time")

# Standardize the latest year data
z_scores_latest = (region_latest - mean_ref) / std_ref
z_flat = z_scores_latest.values.flatten()

# Frequency counts
below_minus_2sd = np.sum(z_flat < -2)
above_2sd = np.sum(z_flat > 2)
total_points = np.sum(~np.isnan(z_flat))  # total valid points

# Output
print(f"Latest year: {latest_year}")
print(f"Total valid points: {total_points}")
print(f"Points < mean - 2*std: {below_minus_2sd}")
print(f"Points > mean + 2*std: {above_2sd}")

# Plot KDE
plt.figure(figsize=(12, 6))
sns.kdeplot(z_flat, color='green', label=f"{latest_year}", linewidth=2)
plt.axvline(x=-2, color='blue', linestyle='--', label='-2σ')
plt.axvline(x=2, color='red', linestyle='--', label='+2σ')
plt.title(f"Standard Normal Distribution of Temperature Z-scores\n({state_name})", fontsize=16)
plt.xlabel("Z-Score", fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.xlim(-4, 4)
plt.show()


C:\Users\Mridul\AppData\Local\Temp\ipykernel_9528\1476995667.py:14: DeprecationWarning: Usage of 'use_cftime' as a kwarg is deprecated. Please pass a 'CFDatetimeCoder' instance initialized with 'use_cftime' to the 'decode_times' kwarg instead.
Example usage:
    time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
    ds = xr.open_dataset(decode_times=time_coder)

  data = xr.open_dataset(file_dir, engine='netcdf4', use_cftime=True)


KeyError: 'State_Name'